# Add web search ability to research assistant

In [4]:
! pip install "pyautogen>=0.2.28" "tavily-python"

In [8]:
## import required libraries
import os
from typing import Annotated
from tavily import TavilyClient
from autogen import AssistantAgent, UserProxyAgent, config_list_from_json, register_function
from dotenv import load_dotenv

load_dotenv(override=True)

True

In [16]:
## llm configuration
llm_config = {
    "model": "gpt-4o",
    "api_key": os.getenv("OPENAI_API_KEY")
}

## Tavily as a tool for searching the web

In [10]:
tavily=TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))

def search_tool(query: Annotated[str, "The search query"]) -> Annotated[str, "The search results"]:
    return tavily.get_search_context(query=query, search_depth="advanced")

## Adding a ReAct prompt

In [11]:
ReAct_prompt = """
Answer the following questions as best you can. You have access to tools provided.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take
Action Input: the input to the action
Obsevation: the result of the action
... (this process can repeat multiple times)
Thought: I now know the final answer
Final answer: the final answer to the original input question

Begin!
Qustion: {input}
"""

# Define the ReAct prompt message. Assuming a "question" field is present in the context
def react_prompt_message(sender, recipient, context):
    return ReAct_prompt.format(input=context["question"])

## Define agnets

In [17]:
user_proxy = UserProxyAgent(
    name="User",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endwith("TERMINATE"),
    human_input_mode="ALWAYS",
    max_consecutive_auto_reply=10,
    code_execution_config=False
)

research_assistant = AssistantAgent(
    name="Assistant",
    system_message="""
                You are a helpful research assistant who has the ability to search the web using the provided tools.
                Only use the tools you have been provided with. Reply TERMINATE when the task is done.
                """,
    llm_config=llm_config,
)

## Register the search tool

In [18]:
register_function(
    search_tool,
    caller=research_assistant,
    executor=user_proxy,
    name="search_tool",
    description="Search the web for the given query"
)

## Initiate the chat

In [19]:
user_proxy.initiate_chat(
    research_assistant,
    message=react_prompt_message,
    question="Who won the T20 cricket world cup in 2024"
)

User (to Assistant):


Answer the following questions as best you can. You have access to tools provided.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take
Action Input: the input to the action
Obsevation: the result of the action
... (this process can repeat multiple times)
Thought: I now know the final answer
Final answer: the final answer to the original input question

Begin!
Qustion: Who won the T20 cricket world cup in 2024


--------------------------------------------------------------------------------
Assistant (to User):

Question: Who won the T20 cricket world cup in 2024
Thought: The T20 cricket world cup is an international event that occurs periodically, and the winner for 2024 would not be included in the data up to October 2023. I should search for recent information regarding the winner.
Action: Use the search tool to find the winner of the T20 cricket world cup in 2024

User (to Assistant):

***** Response from calling tool (call_42EUE6u63UYiyENEHuhPUKRF) *****
[{"url": "https://en.wikipedia.org/wiki/2023_Cricket_World_Cup_final", "content": "The 2023 Cricket World Cup Final was a One Day International cricket match played at the Narendra Modi Stadium in Ahmedabad, on 19 November 2023 to determine the winner of the 2023 Cricket World Cup. It was played between host nation India and Australia. It was the first time that Ahmedabad hosted a Cricket World Cup final. It was the second time that India and Australia played a World Cup final against each other, after the 2003 final. [...] |  |  |\n| --- | --- |\n| Mitchell Starc bowling against Rohit Sharma | |\n| Event | 2023 Cricket World Cup |\n| | India | Australia | | --- | --- | | India | Australia | | 240 | 241/4 | | 50 overs | 43 overs | | |\n| Australia won by 6 wickets. | |\n| Date | 19 November 2023 |\n| Venue | Narendra Modi Stadium, Ahmedabad |\n| Player of the match \"Player of the Match awards 

User (to Assistant):

***** Response from calling tool (call_wMKmCIkze3zlUPEdmrNfSsH9) *****
[{"url": "https://www.espncricinfo.com/series/icc-men-s-t20-world-cup-2024-1411166/match-schedule-fixtures-and-results", "content": "RESULT\u2022 24th Match, Group B (N) \u2022North Sound, [ICC Men's T20 World Cup](https://www.espncricinfo.com/series/icc-men-s-t20-world-cup-2024-1411166 \"T20 World Cup\")  NAM 72  AUS (5.4/20 ov, T:73) 74/1 Australia won by 9 wickets (with 86 balls remaining) RESULT\u2022 29th Match, Group C (N) \u2022Tarouba, [ICC Men's T20 World Cup](https://www.espncricinfo.com/series/icc-men-s-t20-world-cup-2024-1411166 \"T20 World Cup\")  PNG 95  AFG (15.1/20 ov, T:96) 101/3 Afghanistan won by 7 wickets (with 29 balls remaining) RESULT\u2022 46th Match, Super Eights, Group 2 (N) \u2022Bridgetown, [ICC Men's T20 World Cup](https://www.espncricinfo.com/series/icc-men-s-t20-world-cup-2024-1411166 \"T20 World Cup\")  USA 128  WI (10.5/20 ov, T:129) 130/1 West Indies won by 9 w

User (to Assistant):

***** Response from calling tool (call_wrPOSPI2omAYiVjEGPGFbFCo) *****
[{"url": "https://www.espncricinfo.com/series/icc-men-s-t20-world-cup-2024-1411166/match-schedule-fixtures-and-results", "content": "RESULT\u2022 24th Match, Group B (N) \u2022North Sound, [ICC Men's T20 World Cup](https://www.espncricinfo.com/series/icc-men-s-t20-world-cup-2024-1411166 \"T20 World Cup\")  NAM 72  AUS (5.4/20 ov, T:73) 74/1 Australia won by 9 wickets (with 86 balls remaining) RESULT\u2022 29th Match, Group C (N) \u2022Tarouba, [ICC Men's T20 World Cup](https://www.espncricinfo.com/series/icc-men-s-t20-world-cup-2024-1411166 \"T20 World Cup\")  PNG 95  AFG (15.1/20 ov, T:96) 101/3 Afghanistan won by 7 wickets (with 29 balls remaining) RESULT\u2022 46th Match, Super Eights, Group 2 (N) \u2022Bridgetown, [ICC Men's T20 World Cup](https://www.espncricinfo.com/series/icc-men-s-t20-world-cup-2024-1411166 \"T20 World Cup\")  USA 128  WI (10.5/20 ov, T:129) 130/1 West Indies won by 9 w

ChatResult(chat_id=None, chat_history=[{'content': '\nAnswer the following questions as best you can. You have access to tools provided.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take\nAction Input: the input to the action\nObsevation: the result of the action\n... (this process can repeat multiple times)\nThought: I now know the final answer\nFinal answer: the final answer to the original input question\n\nBegin!\nQustion: Who won the T20 cricket world cup in 2024\n', 'role': 'assistant', 'name': 'User'}, {'content': 'Question: Who won the T20 cricket world cup in 2024\nThought: The T20 cricket world cup is an international event that occurs periodically, and the winner for 2024 would not be included in the data up to October 2023. I should search for recent information regarding the winner.\nAction: Use the search tool to find the winner of the T20 cricket world cup in 2024.\n